In [20]:
import numpy as np
import pandas as pd
import vtk
from vtk.util import numpy_support
from sklearn.neighbors import KDTree
from tqdm import tqdm

In [44]:
# FILE_AXI = './special/data.axi'
# FILE_CSV = './special/data.csv'
# FILE_ELE = './special/data.ele'
# FILE_FACE= './special/data.face'
# FILE_NODE = './special/data.node'
# FILE_OUT = './special/MESH.vtu'

FILE_AXI = './mamedov_ok/data.axi'
FILE_CSV = './mamedov_ok/data.csv'
FILE_ELE = './mamedov_ok/data.ele'
FILE_FACE= './mamedov_ok/data.face'
FILE_NODE = './mamedov_ok/data.node'
FILE_OUT = './mamedov_ok/MESH.vtu'

In [45]:
axi_data = np.loadtxt(FILE_AXI, skiprows=1)
axi_data

array([[-0.06808373,  0.45497257,  0.88789898],
       [ 0.90590912,  0.22172089,  0.36078867],
       [-0.99645406,  0.08406679, -0.00347436],
       ..., 
       [-0.37052788,  0.85764381,  0.35658966],
       [-0.34675869,  0.91542733,  0.20433116],
       [-0.33704918,  0.92164678,  0.19226299]])

In [46]:
ele_data = np.loadtxt(FILE_ELE, skiprows=1)
ele_data, len(ele_data)

(array([[  0.00000000e+00,   6.29950000e+04,   5.98890000e+04,
           6.94940000e+04,   8.47410000e+04,   0.00000000e+00],
        [  1.00000000e+00,   5.08390000e+04,   6.79950000e+04,
           6.75400000e+04,   7.60440000e+04,   0.00000000e+00],
        [  2.00000000e+00,   5.43470000e+04,   6.32380000e+04,
           7.43940000e+04,   7.67480000e+04,   0.00000000e+00],
        ..., 
        [  2.13462100e+06,   3.90230000e+04,   3.03877000e+05,
           3.90570000e+04,   2.03158000e+05,   1.00000000e+00],
        [  2.13462200e+06,   3.03877000e+05,   3.90570000e+04,
           3.24221000e+05,   3.90560000e+04,   1.00000000e+00],
        [  2.13462300e+06,   3.03877000e+05,   3.24221000e+05,
           3.90570000e+04,   2.03158000e+05,   1.00000000e+00]]), 2134624)

In [47]:
face_data = np.loadtxt(FILE_FACE, skiprows=1)
face_data

array([[  0.00000000e+00,   4.18590000e+04,   3.00920000e+04,
          4.18570000e+04,   1.00000000e+00],
       [  1.00000000e+00,   5.10430000e+04,   2.42660000e+04,
          5.10410000e+04,   1.00000000e+00],
       [  2.00000000e+00,   2.59780000e+04,   4.57000000e+04,
          4.79100000e+04,   1.00000000e+00],
       ..., 
       [  1.07997000e+05,   7.88000000e+02,   1.60380000e+04,
          1.60330000e+04,   1.00000000e+00],
       [  1.07998000e+05,   1.01430000e+04,   1.01380000e+04,
          2.05100000e+03,   1.00000000e+00],
       [  1.07999000e+05,   9.69300000e+03,   9.69600000e+03,
          9.69500000e+03,   1.00000000e+00]])

In [48]:
node_data = np.loadtxt(FILE_NODE, skiprows=1)
node_data

array([[  0.00000000e+00,  -1.43881500e+01,   3.91870000e+00,
         -4.06612750e+01],
       [  1.00000000e+00,   1.19105000e+00,   6.01311000e+00,
         -3.25054000e+00],
       [  2.00000000e+00,   9.89775000e-02,   5.46275000e+00,
         -4.43365000e+00],
       ..., 
       [  3.65904000e+05,  -3.90878847e+00,   1.35877235e+01,
         -1.99497879e+01],
       [  3.65905000e+05,   8.60037746e+00,   1.17962969e+01,
         -2.16814136e+01],
       [  3.65906000e+05,   5.17447089e+00,   9.88372735e+00,
         -2.81069578e+01]])

In [49]:
node_data[:,1:]

array([[-14.38815   ,   3.9187    , -40.661275  ],
       [  1.19105   ,   6.01311   ,  -3.25054   ],
       [  0.0989775 ,   5.46275   ,  -4.43365   ],
       ..., 
       [ -3.90878847,  13.5877235 , -19.94978786],
       [  8.60037746,  11.79629688, -21.68141362],
       [  5.17447089,   9.88372735, -28.10695779]])

In [50]:
csv_data= np.loadtxt(FILE_CSV)
csv_data

array([[  0.00000000e+00,   2.35323000e+00,   6.26666000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   1.85996000e+00,   6.30465000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  2.00000000e+00,   1.89206000e+00,   5.99648000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       ..., 
       [  5.47670000e+04,   7.25200000e-03,   9.53785000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  5.47680000e+04,  -3.44800000e-03,   9.57761000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  5.47690000e+04,   7.54238000e+00,   8.80523000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [51]:
csv_data[1,1:4]

array([  1.85996,   6.30465, -20.6577 ])

In [52]:
def create_points(array):
    """Create vtkPoints from double array"""
    vtk_points = vtk.vtkPoints()
    test = numpy_support.numpy_to_vtk(array.ravel(), deep=True, array_type=vtk.VTK_FLOAT)
    test.SetNumberOfComponents(3)
    vtk_points.SetData(test)
    return vtk_points
 
def create_cells(array):
    """Create a vtkCellArray from long array"""
    tmp_ = np.zeros((len(array),1))
    tmp_.fill(4)
    vtk_cells = vtk.vtkCellArray()
    data = np.array(np.c_[tmp_, array], dtype='int64')
    print data
    test = numpy_support.numpy_to_vtkIdTypeArray(data.ravel(), deep=True)
    vtk_cells.SetCells(len(array), test)
    return vtk_cells

In [53]:
points_vtk = create_points(node_data[:,1:])
print points_vtk

vtkPoints (0xb21c080)
  Debug: Off
  Modified Time: 7076
  Reference Count: 1
  Registered Events: (none)
  Data: 0x2515c80
  Data Array Name: Points
  Number Of Points: 365907
  Bounds: 
    Xmin,Xmax: (-23.2351, 22.607)
    Ymin,Ymax: (-6.2802, 27.5669)
    Zmin,Zmax: (-40.75, -2.30217)




In [54]:
numpy_support.vtk_to_numpy(points_vtk.GetData())

array([[-14.38815022,   3.91869998, -40.66127396],
       [  1.19105005,   6.01311016,  -3.25054002],
       [  0.0989775 ,   5.46274996,  -4.43365002],
       ..., 
       [ -3.90878844,  13.58772373, -19.94978714],
       [  8.60037708,  11.79629707, -21.68141365],
       [  5.1744709 ,   9.88372707, -28.10695839]], dtype=float32)

In [55]:
cells_vtk = create_cells(ele_data[:,1:-1])
print cells_vtk

[[     4  62995  59889  69494  84741]
 [     4  50839  67995  67540  76044]
 [     4  54347  63238  74394  76748]
 ..., 
 [     4  39023 303877  39057 203158]
 [     4 303877  39057 324221  39056]
 [     4 303877 324221  39057 203158]]
vtkCellArray (0xb5cd600)
  Debug: Off
  Modified Time: 7083
  Reference Count: 1
  Registered Events: (none)
  Number Of Cells: 2134624
  Insert Location: 10673120
  Traversal Location: 0




In [56]:
numpy_support.vtk_to_numpy(cells_vtk.GetData())

array([     4,  62995,  59889, ..., 324221,  39057, 203158])

In [57]:
torso_tree = KDTree(node_data[:,1:], leaf_size=2)

In [58]:
csv_data_vtk = []

for i in tqdm(xrange(0,9)):
    data_for_write = np.zeros(len(node_data))
    data_for_write[:] = None
    
    for row in csv_data: 
        dist, ind = torso_tree.query(row[1:4].reshape(1, -1))
        data_for_write[ind] = row[i]
    
    temp_= numpy_support.numpy_to_vtk(data_for_write, deep=True, array_type=vtk.VTK_FLOAT)
    temp_.SetName('csv_data_{}_vtk'.format(i))
    csv_data_vtk.append(temp_)

100%|██████████| 9/9 [00:21<00:00,  2.40s/it]


In [59]:
grid = vtk.vtkUnstructuredGrid()
grid.SetPoints(points_vtk)
grid.SetCells(10,cells_vtk)

print grid

vtkUnstructuredGrid (0x24a0c50)
  Debug: Off
  Modified Time: 7127
  Reference Count: 1
  Registered Events: (none)
  Information: 0x26a1220
  Data Released: False
  Global Release Data: Off
  UpdateTime: 0
  Field Data:
    Debug: Off
    Modified Time: 7113
    Reference Count: 1
    Registered Events: (none)
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
  Number Of Points: 365907
  Number Of Cells: 2134624
  Cell Data:
    Debug: Off
    Modified Time: 7116
    Reference Count: 1
    Registered Events: (none)
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
    Copy Tuple Flags: ( 1 1 1 1 1 0 1 1 )
    Interpolate Flags: ( 1 1 1 1 1 0 0 1 )
    Pass Through Flags: ( 1 1 1 1 1 1 1 1 )
    Scalars: (none)
    Vectors: (none)
    Normals: (none)
    TCoords: (none)
    Tensors: (none)
    GlobalIds: (none)
    PedigreeIds: (none)
    EdgeFlag: (none)
  Point Data:
    Debug: Off
    Modified Time: 7115
    Reference Count: 1
    

In [60]:
for i in xrange(0,9):
    grid.GetPointData().AddArray(csv_data_vtk[i])

In [61]:
writer = vtk.vtkXMLUnstructuredGridWriter()
writer.SetFileName(FILE_OUT)
writer.SetInputData(grid)
writer.Write()

1

In [62]:
numpy_support.vtk_to_numpy(grid.GetCells().GetData())[:10]

array([    4, 62995, 59889, 69494, 84741,     4, 50839, 67995, 67540, 76044])